<a href="https://colab.research.google.com/github/hungpham13/Laptop-EDA/blob/main/DS_Project_20211.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!git clone https://github.com/hungpham13/Laptop-EDA.git

Cloning into 'Laptop-EDA'...
remote: Enumerating objects: 238, done.
remote: Counting objects: 100% (238/238), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 238 (delta 142), reused 118 (delta 53), pack-reused 0
Receiving objects: 100% (238/238), 24.55 MiB | 15.85 MiB/s, done.
Resolving deltas: 100% (142/142), done.


**[Guideline](https://husteduvn.sharepoint.com/sites/IntroductiontoDataScience-IT4142E-2021-2022/Class%20Materials/Capstone%20project/DS-Capstone%20project%20guidelines.pdf)**
## Topic description assignment (20th of October, 2021)
[Our project proposal](https://docs.google.com/document/d/16L1Mqy_HtDfhxB0xNXmCZSBNMKazWsYgwA9s1Vueih8/edit?usp=sharing)


## Assginment for each member
### **[On GG Sheet](https://docs.google.com/spreadsheets/d/11ldCYnPJwYuMKBba6jX7qPYnYtekAYjR9YvORww76hE/edit#)**

Distinguish between programming tasks and analytic tasks. 
- Examples of programming tasks: data scraping for source A; implementing algorithm A,
etc.
- Examples of analytic tasks: proposing our subject, selecting the algorithms to be used to
solve this problem, writing parts 1/2/3/4 of the report, writing parts 1/2/3/4 of the presentation, etc.

For each task, provide a detailed contribution of each member to this task:
- e.g.: Programming Task 1- data scraping from source A: member1 Mr X 40% ; member 2 Mrs Y 60% (no more than 2 members /task for programming tasks)...).
- e.g.: Analytic Task 1– proposing our subject: member1 Mr X 40% ; member 2 Mrs Y 30% ; member 2 Mr Z 30% (can be more than 2 members /task for analytic
tasks)...)

## Final assignment (29th of December, 2021)
    
Create 1 directory named with group number, upload on the dedicated assignment on Teams:
- Report: PDF file, no more than 15 pages (not included reference pages / first page / table of contents)
- Code: put in a single .zip file, with a readme.txt file explaining your codes structure, how to run the project – if necessary.
- Video demo of your Python code running in a notebook
- Presentation: PDF file, no more than 10 slides. The first slide MUST contain the (recent) pictures of the students in the group, with student’s name/number

# Materials

1. Scrapy, Selenium, BeautifulSoup – Data scraping
2. Numpy, Pandas, Seaborn – data analysis and visualization 

[Kaggle detailed EDA](https://www.kaggle.com/ekami66/detailed-exploratory-data-analysis-with-python)

[Github DS project](https://github.com/veb-101/Data-Science-Projects)

[Havard Data Science Course](http://cs109.github.io/2015/pages/videos.html)

[Web Scraping with Python (Book)](https://vn1lib.org/book/3515980/5d50aa)

[Web Scraping on Kaggle](https://www.kaggle.com/getting-started?search=web+scraping)

In [32]:
import re
import pandas as pd
import numpy as np
import glob

!pip install python-Levenshtein
!pip install fuzzywuzzy
!pip install textdistance
from difflib import SequenceMatcher
from fuzzywuzzy import fuzz
import textdistance

# Data cleaning - Data preprocessing

In [88]:
def text_facet(df, col, print_all=True,ascending=False):
    if print_all:
        with pd.option_context('display.max_rows', 1000, 'display.max_columns', None):
            print(f"---------- {col} ------------")
            print(df[col].value_counts(dropna=False,ascending=ascending))
    else:
        print(f"---------- {col} ------------")
        print(df[col].value_counts(dropna=False,ascending=ascending))
        print(f"Number of null: {df[col].isna().sum()}")

def get_element_fptshop(id):
    r1 = cleaned_fpt.iloc[id]
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(r1)

def lower_all_string(df):
    '''lower all values in string columns of dataframe df'''
    for c in df.columns:
        if df[c].dtypes == np.object:
            df[c] = df[c].str.lower()

data_path = 'Laptop-EDA/scraped_data/'
all_brands = ['apple','asus','msi','hp','lenovo','dell','acer','lg','microsoft',
              'huawei','avita','gigabyte','fujitsu']

In [9]:
retained_cols = ['Comment_count','Price', 'Rating', 'Rating_count']
mapping_cols = ['Series','Brand','cpu_brand','cpu_code','ram_size',\
                'storage_size','displ_size','displ_rate']
other_cols = ['Battery', 'GPU', 'OS', 'P/N',  'displ_resolution', 'hdmi',\
            'Release_year', 'Surface', 'Weight', 'backlit_keyboard', \
            'lan', 'length', 'ram_upgradable', 'screen_technology',\
            'sd_card', 'storage_extra_slot', 'storage_type', \
            'headphone_jack', 'thickness', 'type_c', 'usb_a', 'width']
all_cols = retained_cols + mapping_cols + other_cols

## Thegioididong - Dienmayxanh
result: no screen technology

In [10]:
def clean_thegioididong(source):
    source.dropna(subset=['ports'],inplace=True)
    result = pd.DataFrame()
    result['P/N'] = source['title'].str.extract(r"\(([^)]+)\)",expand=False).str.strip().str.replace("\.$", "") #replace any dot at the end of P/N

    result['Rating'] = source['rating'].fillna(0).astype('float64')
    result['Rating_count'] = source['review_count'].str.extract(r'(\d+)',expand=False).str.strip().fillna(0).astype('int64')

    result['Comment_count'] = 0
    for i,r in source['cmt_count'].iteritems():
        if pd.isnull(r):
            continue
        for c in r.split("|"):
            if re.search(r'([\d.,]+)',c):
                result['Comment_count'][i] += int(re.search(r'([\d.,]+)',c).group().replace(".",""))
    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #     print(result['Comment_count'].value_counts(dropna=False))

    result['Brand'] = source['title'].str.split().str[1].str.strip().str.lower()
    # print(result['Brand'].value_counts(dropna=False))
    result['Series'] = source['title'].str.extract(r'\w+\s\w+\s([\w\s]+)\s[\w\/d]+',expand=False).str.strip()
    result.loc[(result['Series'].str.split().str[-1].str.len() == 2), 'Series'] =  result['Series'].str.slice(stop=-3)

    #add P/N to Series (except Apple laptop)
    result['Series'] = np.where(result['Brand'] != 'apple', 
                                result[['Series', 'P/N']].fillna('').agg(' '.join, axis=1),
                                result['Series'])
    result.Series[result.Series == 'MacBook Pro'] = 'MacBook Pro M1 2020'
    for a in zip(result.Series, source.title):
        print('{0:40} {1}'.format(a[0],a[1]))

    result['Price'] = source['price'].str.extract(r'([\d.]+)',expand=False).str.replace(".","").astype('float64')

    result['cpu_brand'] = source['cpu_name'].str.extract(r'([\w]+)\s',expand=False).str.strip()
    result['cpu_code'] = source['cpu_name'].str.extract(r'-([\w\d\s]+)',expand=False).str.strip().fillna('m1')
    result['cpu_code'] = np.where(result['cpu_code'].str.len() > 10, 
                                    source['cpu_name'].str.extract(r'([\w\d\s]+)-',expand=False).str.split().str.slice(start=1).str.join(' '),
                                    result['cpu_code'])

    result['ram_size'] = source['ram_size'].str.extract(r'(\d+)',expand=False).str.strip().fillna(8).astype('float64')
    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #     print(result['ram_size'].value_counts(dropna=False))

    result['ram_upgradable'] = np.where(source['ram_upgradable'].str.contains("Kh"), "no", "yes")

    result['GPU'] = source['gpu'].str.extract(r'-([\w\d\s-]+),?',expand=False).str.strip()

    result['Battery'] = source['battery'].str.extract(r'([\d\s.]+)Wh',expand=False).str.strip().astype('float64')

    result['storage_size'] = np.nan
    result['storage_type'] = np.nan
    result['storage_extra_slot'] = np.nan
    # for i,r in result.loc[result['storage_size'].notnull(),'storage_size'].iteritems():
    for i,r in source['storage'].iteritems():
        if pd.isnull(r):
            continue
        for s in r.split("|"):
            s = s.lower()
            if "hỗ trợ" not in s:
                result['storage_size'][i] = re.search(r'(\d+)\s?[gtb]{2}',s.strip()).group(1)
                if "ssd" in s:
                    result['storage_type'][i] = "SSD"
                elif "hdd" in s:
                    result['storage_type'][i] = "HDD"
            elif "không" not in s: #just have "ho tro", no "khong"
                result['storage_extra_slot'][i] = 'yes'
        if pd.isnull(result['storage_extra_slot'][i]):
                result['storage_extra_slot'][i] = 'no'
    result['storage_size'] = np.where(result['storage_size']==1,1024,result['storage_size'])
    result['storage_size'] = np.where(result['storage_size']==2,2048,result['storage_size'])
    # print(result['storage_size'].value_counts(dropna=False))

    result['displ_size'] = source['displ_size'].str.extract(r'([\d.]+)',expand=False).astype('float64')
    result['displ_resolution'] = source['displ_resolution'].str.extract(r"\(([^)]+)\)",expand=False).str.strip()
    result['displ_rate'] = source['displ_rate'].str.extract(r'([\d]+)',expand=False).fillna(60).astype('int64')
    result['screen_technology'] = np.nan
    for i,r in source.displ_tech.iteritems():
        if pd.isnull(r):
            continue
        elif 'ips' in r.lower():
            result['screen_technology'][i] = 'IPS'
        elif 'oled' in r.lower():
            result['screen_technology'][i] = 'OLED'
        elif 'wva' in r.lower():
            result['screen_technology'][i] = 'VA'
    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #     print(result['screen_technology'].value_counts(dropna=False))
    #     print(source.displ_tech[result.screen_technology.isnull()])

    result['Weight'] = source['weight_dims'].str.extract(r"Nặng ([\d.]+)",expand=False).str.strip().astype('float64')
    result['length'] = source['weight_dims'].str.extract(r"Dài ([\d.]+)",expand=False).str.strip().astype('float64')
    result['width'] = source['weight_dims'].str.extract(r"Rộng ([\d.]+)",expand=False).str.strip().astype('float64')
    result['thickness'] = source['weight_dims'].str.extract(r"Dày ([\d.]+)",expand=False).str.strip().astype('float64')

    result['OS'] = source['os'].str.split().str[:2].str.join(" ").str.strip()
    result['Surface'] = np.nan
    for i,r in source['surface'].iteritems():
        if pd.isnull(r):
            continue
        d = {"nhôm":"Aluminum","carbon":"Carbon","kim loại": "Metal", "nhựa":"Plastic","magie":"Magnesium"}
        s = []
        for vn_term in d:
            if vn_term in r.lower():
                s.append(d[vn_term])
        result['Surface'][i] = ', '.join(s)
        
    result['Release_year'] = source['launch']
    result['backlit_keyboard'] = np.where(source['backlight_key'].str.contains('Không'),'no','yes')

    result['hdmi'] = np.where(source['ports'].str.lower().str.contains("hdmi"),'yes','no')
    result['headphone_jack'] = np.where(source['ports'].str.lower().str.contains("jack"),'yes','no')
    result['lan'] = np.where(source['ports'].str.lower().str.contains("lan"),'yes','no')
    result['type_c'] = np.where(source['ports'].str.lower().str.contains("-c"),'yes','no')
    result['usb_a'] = np.where(source['ports'].str.lower().str.contains(r"usb [a32]",regex=True),'yes','no')
    result['sd_card'] = np.where(pd.isnull(source['sd_card']),'no','yes')
    lower_all_string(result)
    return result

In [11]:
scraped_tgdd = pd.read_csv(data_path+"tgdd_dmx.csv",index_col='url')
cleaned_tgdd = clean_thegioididong(scraped_tgdd)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(cleaned_tgdd.Series.value_counts(dropna=False))
#     print(cleaned_tgdd)
cleaned_tgdd.to_csv(data_path+'tgdd_dmx_cleaned.csv',encoding='utf-8')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: Settin

Katana Gaming GF66 11UC 224VN            Laptop MSI Katana Gaming GF66 11UC i7 11800H/8GB/512GB/4GB RTX3050/144Hz/Balo/Win10 (224VN)
Gaming Pulse GL66 11UDK 816VN            Laptop MSI Gaming Pulse GL66 11UDK i7 11800H/16GB/512GB/4GB RTX3050Ti/144Hz/Balo/Chuột/Win10 (816VN)
Nitro 5 Gaming AN515 57 727J NH.QD9SV.005 Laptop Acer Nitro 5 Gaming AN515 57 727J i7 11800H/8GB/512GB/4GB RTX3050Ti/144Hz/Win10 (NH.QD9SV.005.)
Nitro 5 Gaming AN515 57 5831 NH.QDGSV.003 Laptop Acer Nitro 5 Gaming AN515 57 5831 i5 11400H/8GB/512GB/6GB RTX3060/144Hz/Win10 (NH.QDGSV.003)
Aspire 7 Gaming A715 42G R6ZR NH.QAYSV.003 Laptop Acer Aspire 7 Gaming A715 42G R6ZR R5 5500U/8GB/512GB/4GB GTX1650/144Hz/Win10 (NH.QAYSV.003)
Nitro 5 Gaming AN515 57 720A NH.QEQSV.004 Laptop Acer Nitro 5 Gaming AN515 57 720A i7 11800H/8GB/512GB/4GB RTX3050Ti/144Hz/Win11 (NH.QEQSV.004)
Gaming G5 KC-5S11130SB                   Laptop Gigabyte Gaming G5 i5 10500H/16GB/512GB/6GB RTX3060/144Hz/Win11 (KC-5S11130SB)
Nitro 5 Gaming AN515 57 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:83:

## Fpt shop
- Rename column to the same format as data form
- Clean `Series`,`displ_rate`
- `Rating`,`Rating_count`: (5,1) to (0,0)

In [12]:
scraped_fpt = pd.read_csv(data_path+"fptshop_scraped.csv")
cleaned_fpt = pd.read_csv(data_path+"fpt-cleaned.csv")
cleaned_fpt.rename(columns={'PN':'P/N',
                            'rating':'Rating',
                            'rating_count':'Rating_count',
                            'comment_count':'Comment_count',
                            'price':'Price',
                            'RAM':'ram_size',
                            'RAM_upgradable':'ram_upgradable',
                            'battery_wh':'Battery',
                            'storage_upgradable':'storage_extra_slot',
                            'screen_size':'displ_size',
                            'display_resolution':'displ_resolution',
                            'display_rating':'displ_rate',
                            'weight':'Weight',
                            'height':'thickness',
                            'surface_material':'Surface',
                            'release_year':'Release_year', 
                            'jack':'headphone_jack',
                            'usb':'usb_a', 
                            'sd':'sd_card',
                            'keyboard_light':'backlit_keyboard'
                            },inplace=True)

lower_all_string(cleaned_fpt)
cleaned_fpt['P/N'] = cleaned_fpt['P/N'].fillna('').astype('object')
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     scraped_fpt.info()
#     cleaned_fpt.info()
#     print(cleaned_fpt['Series'].value_counts(dropna=False))
for i,row in cleaned_fpt.iterrows():
    r = row.Series
    extract_r1 = re.search(r'([\w\s-]+)[ /]c?o?r?e?\s?[ir\d]{2}[- ]',r)
    extract_r2 = re.search(r'([\w\s-]+)pentium',r)
    extract_r3 = re.search(r'([\w\s-]+)celeron',r)
    extract_r4 = re.search(r'([\w\s-]+)n4020',r)
    extract_r5 = re.search(r'([\w\s-]+)/n4500',r)
    if extract_r1: extract_r = extract_r1.group(1).strip()
    elif extract_r2: extract_r = extract_r2.group(1).strip()
    elif extract_r3: extract_r = extract_r3.group(1).strip()
    elif extract_r4: extract_r = extract_r4.group(1).strip()
    elif extract_r5: extract_r = extract_r5.group(1).strip()
    else: extract_r = r.lower().strip()

    specs_key = [r'\d+gb',r'[\d.]+ghz',r'\d+gpu',r'\d+tb','core i5',r'\d+"']
    for k in specs_key:
        extract_r = re.sub(k, '', extract_r).strip(' -/')
        extract_r = ' '.join(extract_r.split())
    #add P/N
    if row['P/N'] not in extract_r and row.Brand != 'apple':
        extract_r = f"{extract_r} {row['P/N']}"

    print(f"{r:55}  {row['P/N']:20}  {extract_r}")
    cleaned_fpt.Series[i] = extract_r

#disp_rate
cleaned_fpt.displ_rate = cleaned_fpt.displ_rate.replace("--",60).fillna(60).astype('int64')
print(cleaned_fpt.displ_rate.value_counts(dropna=False))

#rating
cleaned_fpt.Rating[cleaned_fpt.Rating_count == 1] = 0
cleaned_fpt.Rating_count[cleaned_fpt.Rating_count == 1] = 0
cleaned_fpt.Rating_count = cleaned_fpt.Rating_count.fillna(0).astype('int64')

#Battery
cleaned_fpt.Battery = cleaned_fpt.Battery

#screen_tech
for i,r in source.displ_tech.iteritems():
    if pd.isnull(r):
        continue
    elif 'ips' in r.lower():
        result['screen_technology'][i] = 'IPS'
    elif 'oled' in r.lower():
        result['screen_technology'][i] = 'OLED'
    elif 'wva' in r.lower():
        result['screen_technology'][i] = 'VA'

#cpu_code
cleaned_fpt.cpu_code.fillna('i5',inplace=True)

nitro gaming an515-57-74nu/i7-11800h                     nh.qd9sv.001          nitro gaming an515-57-74nu nh.qd9sv.001
modern 14 b11mou 852vn i5 1155g7/852vn                   852vn                 modern 14 b11mou 852vn
inspiron n3511 i3 1115g4/p112f001abl                     p112f001              inspiron n3511 p112f001
vostro v3500 i3 1115g4                                   v5i3001w              vostro v3500 v5i3001w
ideapad slim 3 15itl6 i3 1115g4                          82h80040vn            ideapad slim 3 15itl6 82h80040vn
inspiron n3511 i5 1135g7/p112f001bbl                     p112f001              inspiron n3511 p112f001
ideapad slim 5 15itl05 i5 1135g7                         82fg00r0vn            ideapad slim 5 15itl05 82fg00r0vn
gaming gf63 thin 10sc-481vn i7 10750h                    481vn                 gaming gf63 thin 10sc-481vn
inspiron n5515 r5 5500u/p106f003asl                      p106f003asl           inspiron n5515 p106f003asl
thinkbook 14 g3 acl r7 5700u/21a20

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [73]:
for c in cleaned_fpt.columns:
    text_facet(cleaned_fpt,c)

---------- P/N ------------
2019                3
2020                2
nk                  2
2021                2
p112f001            2
70253897            2
2z6j6pa             1
46m12pa             1
70262927            1
a515ep-bq496t       1
90nr07h3-m00090     1
90nb0rz1-m13430     1
4r0u8pa             1
82ln00apvn          1
70252800            1
90nr05u1-m00940     1
v5i3001w            1
g5 md-51s1123sh     1
518v7pa             1
036vn               1
481vn               1
089vn               1
x515ea-bq1004t      1
1a2f1av             1
46m00pa             1
fx706hc-hx009t      1
852vn               1
p106f003asl         1
mk183sa/a           1
90nb0rz2-m12520     1
90nb0s41-m02460     1
nh.qbcsv.001        1
82l3005fvn          1
171n3pa             1
292vn               1
20xw0077vn          1
mwp42sa/a           1
nh.qaysv.004        1
nh.qdqsv.001        1
061vn               1
z11b                1
90nx0sx1-m10080     1
82n7002lvn          1
n3501c              1
014v

## Cellphone S
- Drop column `'1star', '2star', '3star', '4star', '5star'`
- Rename column `comment_count`
- Change Price to float
- recleaned `Series`, `displ_size`

In [13]:
scraped_cellphones = pd.read_csv(data_path+"cellphones_scraped.csv",index_col='url')
# cleaned_cellphones2 = pd.read_csv("Laptop-EDA/scraped_data/cellphones_cleaned.csv",index_col='url')
cleaned_cellphones = pd.read_csv(data_path+"cellphones-cleaned.csv",index_col='url')

cleaned_cellphones.drop(columns=['1star', '2star', '3star', '4star', '5star'],inplace=True)
cleaned_cellphones.rename(columns={'comment_count':'Comment_count',
},inplace=True)
cleaned_cellphones['Price'] = cleaned_cellphones['Price'].str.extract(r'([\d.]+)',expand=False).str.replace(".","").astype('float64')
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     scraped_cellphones.info()
#     cleaned_cellphones.info()
#     cleaned_cellphones2.info()
#     # print(scraped_cellphones['Series'].value_counts(dropna=False))
# cleaned_cellphones.head()

for i,cleaned_row in cleaned_cellphones.Series.iteritems():
    r = scraped_cellphones.item_name[i]
    if not isinstance(r, str):
        # print(r)
        r = r[0]
    black_key = [r'\d+gb',r'\d+inch',r'\d+gpu',r'\d+tb',
                 'laptop','chính','hãng','việt','nam','nhập','khẩu','cũ','đẹp','bạc','mqd32',
                 'apple','asus','msi','hp','lenovo','dell','acer','lg','microsoft','huawei','avita','gigabyte']
    cpu_key = ['core','ryzen']
    specs_key = [r'\d+\sgb',r'[\d.]+\sinch',r'\d+\sgpu',r'\d+\stb',
                 'i\d [\d.]{3}','\d+ cpu -','\([\w\d]+\)','\dk ','radeon.+']
    extract_r1 = []
    for index,w in enumerate(r.lower().split()):
        if w in cpu_key:
            break
        if not any(bool(re.match(k,w)) for k in black_key) or (w == 'laptop' and index!=0):
            extract_r1.append(w)
    extract_r1 = ' '.join(extract_r1).strip(' -')
    for k in specs_key:
        extract_r1 = re.sub(k, '', extract_r1).strip(' -')
    extract_r1 = ' '.join(extract_r1.split())
    # extract_r2 = re.search(r'([\w\s-]+)',r.lower())
    # print('{0:80} {1}'.format(r,extract_r1))
    print(f'{r:70} {cleaned_row:20} {extract_r1}')
    cleaned_cellphones.Series[i] = extract_r1

cleaned_cellphones.dropna(subset=['storage_size','displ_size'],inplace=True)
cleaned_cellphones.Rating.fillna(0,inplace=True)
#recleaned displ_size
cleaned_cellphones.displ_size.loc['https://cellphones.com.vn/laptop-asus-rog-strix-g17-g713qr-hg073t.html'] = 17.3
cleaned_cellphones.displ_size.loc['https://cellphones.com.vn/laptop-hp-15-dy2093dx-405f7ua.html'] = 15.6
#lower all string attributes
lower_all_string(cleaned_cellphones)
#cpu_code fill na
for i,r in cleaned_cellphones.cpu_code[cleaned_cellphones.Brand == 'apple'].iteritems():
    if pd.isnull(r):
        for key in ['i5','i7','m1']:
            name = scraped_cellphones['item_name'][i]
            cpu = scraped_cellphones['Loại CPU'][i]
            if not isinstance(name,str):
                name = name[0]
            if not isinstance(cpu,str):
                cpu = cpu[0]
            if key in name.lower() or key in cpu.lower():
                cleaned_cellphones.cpu_code[i] = key
scraped_cellphones = pd.read_csv(data_path+"cellphones_scraped.csv",index_col='url')
# cleaned_cellphones2 = pd.read_csv("Laptop-EDA/scraped_data/cellphones_cleaned.csv",index_col='url')
cleaned_cellphones = pd.read_csv(data_path+"cellphones-cleaned.csv",index_col='url')

cleaned_cellphones.drop(columns=['1star', '2star', '3star', '4star', '5star'],inplace=True)
cleaned_cellphones.rename(columns={'comment_count':'Comment_count',
},inplace=True)
cleaned_cellphones['Price'] = cleaned_cellphones['Price'].str.extract(r'([\d.]+)',expand=False).str.replace(".","").astype('float64')
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     scraped_cellphones.info()
#     cleaned_cellphones.info()
#     cleaned_cellphones2.info()
#     # print(scraped_cellphones['Series'].value_counts(dropna=False))
# cleaned_cellphones.head()

for i,cleaned_row in cleaned_cellphones.Series.iteritems():
    r = scraped_cellphones.item_name[i]
    if not isinstance(r, str):
        # print(r)
        r = r[0]
    black_key = [r'\d+gb',r'\d+inch',r'\d+gpu',r'\d+tb',
                 'laptop','chính','hãng','việt','nam','nhập','khẩu','cũ','đẹp','bạc','mqd32',
                 'apple','asus','msi','hp','lenovo','dell','acer','lg','microsoft','huawei','avita','gigabyte']
    cpu_key = ['core','ryzen']
    specs_key = [r'\d+\sgb',r'[\d.]+\sinch',r'\d+\sgpu',r'\d+\stb',
                 'i\d [\d.]{3}','\d+ cpu -','\([\w\d]+\)','\dk ','radeon.+']
    extract_r1 = []
    for index,w in enumerate(r.lower().split()):
        if w in cpu_key:
            break
        if not any(bool(re.match(k,w)) for k in black_key) or (w == 'laptop' and index!=0):
            extract_r1.append(w)
    extract_r1 = ' '.join(extract_r1).strip(' -')
    for k in specs_key:
        extract_r1 = re.sub(k, '', extract_r1).strip(' -')
    extract_r1 = ' '.join(extract_r1.split())
    # extract_r2 = re.search(r'([\w\s-]+)',r.lower())
    # print('{0:80} {1}'.format(r,extract_r1))
    print(f'{r:70} {cleaned_row:20} {extract_r1}')
    cleaned_cellphones.Series[i] = extract_r1

cleaned_cellphones.dropna(subset=['storage_size','displ_size'],inplace=True)
cleaned_cellphones.Rating.fillna(0,inplace=True)
#recleaned displ_size
cleaned_cellphones.displ_size.loc['https://cellphones.com.vn/laptop-asus-rog-strix-g17-g713qr-hg073t.html'] = 17.3
cleaned_cellphones.displ_size.loc['https://cellphones.com.vn/laptop-hp-15-dy2093dx-405f7ua.html'] = 15.6
#lower all string attributes
lower_all_string(cleaned_cellphones)
#cpu_code fill na
for i,r in cleaned_cellphones.cpu_code[cleaned_cellphones.Brand == 'apple'].iteritems():
    if pd.isnull(r):
        for key in ['i5','i7','m1']:
            name = scraped_cellphones['item_name'][i]
            cpu = scraped_cellphones['Loại CPU'][i]
            if not isinstance(name,str):
                name = name[0]
            if not isinstance(cpu,str):
                cpu = cpu[0]
            if key in name.lower() or key in cpu.lower():
                cleaned_cellphones.cpu_code[i] = key

cleaned_cellphones.cpu_code.loc['https://cellphones.com.vn/macbook-pro-14-inch-2021-1tb.html'] = 'm1 pro'
cleaned_cellphones.cpu_code.loc['https://cellphones.com.vn/macbook-pro-16-inch-2021-m1-max.html'] = 'm1 max'

Apple MacBook Air 13 inch 128GB MQD32 I Chính hãng Apple Việt Nam      MacBook Air          macbook air i
Laptop Dell Inspiron 3505 Y1N1T2                                       Inspiron             inspiron 3505 y1n1t2
Huawei Matebook 14                                                     Matebook             matebook 14
Huawei Matebook X                                                      Matebook X           matebook x
iMac 24 2021 M1 7GPU 8GB 256GB I Chính hãng Apple Việt Nam             iMac                 imac 24 2021 m1 i
Apple iMac 24 2021 M1 7GPU 16GB 256GB Chính Hãng                       iMac                 imac 24 2021 m1
iMac 24 2021 M1 8GPU 8GB 256GB I Chính hãng Apple Việt Nam             iMac                 imac 24 2021 m1 i
iMac 24 2021 M1 8GPU 16GB 256GB I Chính hãng Apple Việt Nam            iMac                 imac 24 2021 m1 i
iMac 24 2021 M1 8GPU 16GB 512GB I Chính hãng Apple Việt Nam            iMac                 imac 24 2021 m1 i
iMac 24 2021 M1 8GPU 8GB 5

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Laptop HP 15S-FQ2558TU 46M26PA                                                          nan 15s-fq2558tu 46m26pa
Laptop HP 15S-FQ2561TU 46M29PA                                                          nan 15s-fq2561tu 46m29pa
Laptop HP 15S-FQ2602TU 4B6D3PA                                                          nan 15s-fq2602tu 4b6d3pa
Laptop HP 15 Z-EF2000 2J4V7AV Nhập khẩu chính hãng                                      nan 15 z-ef2000 2j4v7av
Laptop HP 240 G8 342A3PA                                                                nan 240 g8 342a3pa
Laptop HP 240 G8 342G5PA                                                                nan 240 g8 342g5pa
Laptop HP 240 G8 342G7PA                                                                nan 240 g8 342g7pa
Laptop HP 240 G8 3D0E8PA                                                                nan 240 g8 3d0e8pa
Laptop HP 240 G8 3D3H6PA                                                                nan 240 g8 3d3h6pa
Laptop HP 240 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Apple MacBook Air 13 inch 128GB MQD32 I Chính hãng Apple Việt Nam      MacBook Air          macbook air i
Laptop Dell Inspiron 3505 Y1N1T2                                       Inspiron             inspiron 3505 y1n1t2
Huawei Matebook 14                                                     Matebook             matebook 14
Huawei Matebook X                                                      Matebook X           matebook x
iMac 24 2021 M1 7GPU 8GB 256GB I Chính hãng Apple Việt Nam             iMac                 imac 24 2021 m1 i
Apple iMac 24 2021 M1 7GPU 16GB 256GB Chính Hãng                       iMac                 imac 24 2021 m1
iMac 24 2021 M1 8GPU 8GB 256GB I Chính hãng Apple Việt Nam             iMac                 imac 24 2021 m1 i
iMac 24 2021 M1 8GPU 16GB 256GB I Chính hãng Apple Việt Nam            iMac                 imac 24 2021 m1 i
iMac 24 2021 M1 8GPU 16GB 512GB I Chính hãng Apple Việt Nam            iMac                 imac 24 2021 m1 i
iMac 24 2021 M1 8GPU 8GB 5

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


## Laptop Arena


In [92]:
la_mapping_cols = ['Brand','Family','Series','Model name','Part Number',
                   'Processor manufacturer','Processor model','Internal memory',
                   'Processor series','Processor family','Release Date',
                   'Total storage capacity','Display diagonal','Maximum refresh rate']
def import_laptoparena(data_path):
    frames = []
    for file_path in glob.glob(data_path+'laptoparena*'):
        df = pd.read_csv(file_path,index_col='product_url')
        frames.append(df)
    df = pd.concat(frames)
    return df.reindex(sorted(df.columns),axis=1)

scraped_laptoparena = import_laptoparena(data_path)
print(len(scraped_laptoparena.columns))


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (8,9,12,15,16,22,27,31,38,40,43,44,49,56,58,61,63,65,67,69,70,73,74,75,76,77,80,85,86,87,89,91,96,103,106,110,111,113,114,123,126,135,136,142,143,144,147,153,155,164,167,168,170,172,174,176,180,181,187,188,190,194,196,199,204,207,208,214,216,219,220,225,230,231,237,239,242,249,250,252,255,256,258,260,261,262,266,269,271,272,273,274,276,278,288,289,295,300,308,311,324,328,334,341,345,346,347,351,357,358,364,370,371,375,376,377,378,379,381,383,387,393,397,398,399,402,404,405,408,409,410,411,416,417,419,421,425,427,429,433,434,437,438) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0,1,2,4,6,7,8,9,11,12,18,20,23,24,29,35,39,41,42,46,47,48,49,50,54,60,61,68,69,72,75,76,79,82,87,89,91,93,94,98,99,100,102,104,106,117,130,134,136,137,13

604


In [115]:
print('Width: ', len(scraped_laptoparena.columns))
print('Length: ',len(scraped_laptoparena))
key_word = []
for c in ['Family','Series']:
    key_word.extend(scraped_laptoparena[c].fillna('').str.lower().tolist())
    # text_facet(laptoparena_scrapped,c,print_all=True)
key_word = list(set(" ".join(key_word).split()))

Width:  604
Length:  77464


In [18]:
def remove_duplicate(l):
    return list(dict.fromkeys(l))

def clean_laptoparena(source,use_part_number=True):
    result = pd.DataFrame()

    result['Brand'] = source['Brand'].str.strip()

    series = source[['Family','Series','Model name']]
    series['product_title'] = source['product_title'].str.split().str[1:].str.join(" ").str.split(' - ').str[0]
    result['Series'] = series.fillna('').agg(' '.join, axis=1).str.split().apply(remove_duplicate).str.join(" ")
    text_facet(result,'Series',print_all=False)

    result['cpu_brand'] = source['Processor manufacturer'].str.strip().str.lower()
    source[['Processor family','Processor series']] = source[['Processor family','Processor series']].fillna('')
    for i,r in source[source['Processor manufacturer'].isnull()].iterrows():
        row = ' '.join(r[['Processor family','Processor series']]).lower()
        for chip in ('intel','qualcomm','amd'):
            if chip in row:
                result.cpu_brand[i] = chip

    # text_facet(source,'Processor model',print_all=False)
    result['cpu_code'] = source['Processor model'].str.split('-').str[-1].str.strip()
    # text_facet(result,'cpu_code',print_all=False)

    # text_facet(source,'Internal memory',print_all=False)
    result['ram_size'] = source['Internal memory'].str.extract(r'(\d+)',expand=False).str.strip().astype('float64')
    # text_facet(result,'ram_size',print_all=False)

    # text_facet(source,'Total storage capacity',print_all=False)
    result['storage_size'] = source['Total storage capacity'].str.extract(r'(\d+)',expand=False).str.strip().astype('float64')
    result['storage_size'][result['storage_size'] == 1000] = 1024
    # text_facet(result,'storage_size',print_all=False)

    # text_facet(source,'Display diagonal',print_all=False)
    result['displ_size'] = source['Display diagonal'].str.extract(r'([\d.]+)',expand=False).str.strip().astype('float64')
    # text_facet(result,'displ_size',print_all=False)

    # text_facet(source,'Maximum refresh rate',print_all=False)
    result['displ_rate'] = source['Maximum refresh rate'].str.extract(r'([\d.]+)',expand=False).str.strip().fillna(60).astype('int64')
    # text_facet(result,'displ_rate',print_all=False)
    result['Release_date'] = pd.to_datetime(source['Release Date'])
    lower_all_string(result)


    result.dropna(subset=mapping_cols+['Release_date'],inplace=True)
    # for c in result.columns:
    #     if len(result[result[c].isnull()]) > len(result)/2:
    #         result.drop(columns=c,inplace=True)
    #         continue
    #     else:
    #         text_facet(result,c,print_all=False)
    result = result[result.Brand.isin(all_brands)]
    return result

cleaned_laptoparena = clean_laptoparena(laptoparena_scrapped)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


---------- Series ------------
ThinkPad X X1 Carbon                604
IdeaPad 300 330                     538
Latitude 5000 E5470                 482
Surface Laptop 3                    457
Surface Laptop 4                    454
                                   ... 
VivoBook Flip TM420IA-EC070T          1
U series 14U360-LR10K                 1
Spectre x360 13 13-aw0196tu           1
Gaming GP GP73 8RE-033FR Leopard      1
TUF Gaming FA506IV-AL127T             1
Name: Series, Length: 38363, dtype: int64
Number of null: 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
print(cleaned_laptoparena.cpu_code[cleaned_laptoparena.cpu_code.isnull()])

In [ ]:
with pd.option_context('display.max_columns', None):
    print(scraped_laptoparena[['Processor manufacturer','Processor series','Processor family']][scraped_laptoparena['Processor manufacturer'].isnull()])
# for c in .columns:
#     if 'processor' in c.lower():
#         text_facet(laptoparena,c)

# Integrate
cleaned data are saved as `cleaned_tgdd`,`cleaned_fpt`,`cleaned_cellphones`

TODO
- Chọn các attribute và xử lý các attribute đó trong cả 3 file - Series ✅
- Kết nối 3 data của tgdd, cellphone và fpt bao gồm các mapping attribute và hợp nhất các retained attribute ✅ (cách hợp nhất các retained phải xem lại)
- Kết nối data thu được ở bước 2 với data của laptop arena dựa vào các mapping attribute, giữ lại các retained.
- Xử lý data đã map được ở bước 3 (loại bỏ các cột quá nhiều null, format dữ liệu, điền tay cho các máy của apple)


## Check out mapping attribute in TGDD, FPT and CellphoneS
- Mapping attribute must be non-null, except for Apple brand ✅
- Retained attribute type is float or int ✅
- Three dataset must have same set of column ✅
- No duplicated rows ✅

In [ ]:
cleaned_fpt[['Series','Brand','cpu_brand','cpu_code','ram_size','storage_size',
             'displ_size','displ_rate',
             'Price','Rating','Rating_count','Comment_count']].info()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(cleaned_fpt.Brand[cleaned_fpt.cpu_code.isnull()])
    print(cleaned_fpt[cleaned_fpt['P/N'].isnull()])
cleaned_fpt.head(3)

In [77]:
cleaned_cellphones[all_cols].info()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(cleaned_cellphones.Brand[cleaned_cellphones.Price.isnull()])
    print(cleaned_cellphones.Brand[cleaned_cellphones.ram_size.isnull()])
    print(cleaned_cellphones.Brand[cleaned_cellphones.storage_size.isnull()])
    print(cleaned_cellphones.Brand[cleaned_cellphones.displ_size.isnull()])
cleaned_cellphones.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 416 entries, https://cellphones.com.vn/apple-macbook-air-13-inch-128-gb-mqd32-chinh-hang.html to https://cellphones.com.vn/surface-pro-7-plus.html
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Comment_count       416 non-null    int64  
 1   Price               352 non-null    float64
 2   Rating              416 non-null    float64
 3   Rating_count        416 non-null    int64  
 4   Series              416 non-null    object 
 5   Brand               416 non-null    object 
 6   cpu_brand           416 non-null    object 
 7   cpu_code            416 non-null    object 
 8   ram_size            416 non-null    float64
 9   storage_size        416 non-null    float64
 10  displ_size          416 non-null    float64
 11  displ_rate          416 non-null    int64  
 12  Battery             364 non-null    float64
 13  GPU                 413 non-null   

,sd_card,P/N,ram_upgradable,GPU,displ_rate,backlit_keyboard,Weight,thickness,width,length,screen_technology,Battery,OS,displ_resolution,Surface,displ_size,Price,Comment_count,Rating_count,Rating,ram_size,cpu_code,cpu_brand,storage_size,storage_type,storage_extra_slot,Release_year,Brand,Series,usb_a,type_c,lan,headphone_jack,hdmi
url,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
https://cellphones.com.vn/apple-macbook-air-13-inch-128-gb-mqd32-chinh-hang.html,yes,mqd32,NaN,intel hd 6000,60,NaN,1.35,17,227.0,325.00,NaN,54.0,macos,1440 x 900,NaN,13.3,20000000.0,306,0,0.0,8.0,i5,intel,128.0,ssd,NaN,2017.0,apple,macbook air i,yes,no,no,no,no
https://cellphones.com.vn/aptop-dell-inspiron-15-3505-y1n1t2.html,no,y1n1t2,yes,amd radeon,60,no,1.83,18,249.0,363.96,NaN,42.0,windows 10,1920 x 1080,NaN,15.6,18290000.0,28,0,0.0,8.0,3500u,amd,512.0,ssd,NaN,NaN,dell,inspiron 3505 y1n1t2,yes,no,yes,yes,yes
https://cellphones.com.vn/huawei-matebook-14.html,no,NaN,NaN,nvidia geforce mx350,60,NaN,1.53,15.9,223.8,307.50,NaN,56.0,windows 10,2160 x 1440,NaN,14.0,NaN,3,0,0.0,8.0,10210u,intel,512.0,ssd,NaN,NaN,huawei,matebook 14,no,yes,no,yes,yes


In [ ]:
cleaned_tgdd[['Series','Brand','cpu_brand','cpu_code','ram_size','storage_size','displ_size','displ_rate','Price','Rating','Rating_count','Comment_count']].info()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(cleaned_tgdd.Brand[cleaned_tgdd.cpu_code.isnull()])
cleaned_tgdd.head(3)

In [ ]:
print(sorted(cleaned_fpt.columns))
print(sorted(cleaned_tgdd.columns))
print(sorted(cleaned_cellphones.columns))

In [ ]:
print(f"Duplicated tgdd: {any(cleaned_tgdd.duplicated())}")
print(f"Duplicated cellphones: {any(cleaned_cellphones.duplicated())}")
print(f"Duplicated fpt: {any(cleaned_fpt.duplicated())}")

Rating_count chia làm 2 tập

In [ ]:
# col = 'Rating'
# col = 'Release_year'
# col = 'Comment_count'
# col = 'Brand'
# col = 'storage_type'
for col in mapping_cols:
    for data in (cleaned_cellphones,cleaned_fpt,cleaned_tgdd):
        text_facet(data[data.Brand == 'apple'],col)
# data = cleaned_tgdd
# data[data.Brand == 'apple'].Series

## Merge TGDD, FPT and CellphoneS
Retained attributes:
- Comment = Sum of 3
- Rating Count = Sum of 3
- Availabe_in = Number of existence in stores
- Rating point = Average of `Rating` with weight is `Rating_count`
- Price = Average of `Price`

String similarity
<ol>
<li><a href="http://en.wikipedia.org/wiki/Hamming_distance">Hamming distance</a>  </li>
<li><a href="http://en.wikipedia.org/wiki/Levenshtein_distance">Levenshtein distance</a></li>
<li><a href="http://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance">Damerau–Levenshtein distance</a>  </li>
<li><a href="http://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance">Jaro–Winkler distance</a></li>
</ol>

Jellyfish: https://github.com/jamesturk/jellyfish

In [141]:
'm1' in key_word

False

In [134]:
max(1,2)

2

In [145]:
lev = textdistance.Levenshtein(qval=None)
hamming =  textdistance.Hamming(qval=None)
def similar_str(a, b, algo):
    '''
    Compare string similarity with threshold
    Input: a,b - string, thres - float 
    Output: float
    '''
    if algo == 'normal':
        return SequenceMatcher(None, a, b).ratio()

    elif algo == 'custom':
        both = 0
        a_word, b_word = 0,0
        just_a, just_b = [],[]
        for x in a.split():
            if x in b.split() and x in key_word: both += 1
            if x not in key_word or x not in b.split(): just_a.append(x)
            if x in key_word: a_word += 1
        for x in b.split():
            if x in key_word: b_word += 1
            if x not in a.split() or x not in key_word: just_b.append(x)
        return 0.35*(both/max(a_word,1) + both/max(b_word,1)) + 0.3*fuzz.ratio(" ".join(just_a)," ".join(just_b))/100

    elif algo == 'lev2':
        return lev.normalized_similarity(a,b)
    elif algo == 'lev':
        return fuzz.ratio(a,b)/100

s1 = 'gaming predator helios 300 ph315-54-78w5 nh.qc5sv.001'
s2 = 'Predator Helios 300 PH315-53-781R NH.Q7YAA.001'.lower()
s3 = ('Acer Nitro AN515-51-79DZ'.lower(), 'nitro 5 an515 57 74nu nh.qd9sv.00 1')
s4 = ('macbook air 2020 m1','macbook pro m1 2020')
for i in ('Nitro','5','AN515-51-79DZ','NH.Q2QAA.007'):
    print(similar_str(i.lower(),s3[1],'lev'))
    print(similar_str(i.lower(),s3[1],'custom'))
    print('-----------------------------------------')
print(similar_str(s2,s1,'lev'))
print(similar_str(s2,s1,'custom'))
print(similar_str(s3[0],s3[1],'lev'))
print(similar_str(s3[0],s3[1],'custom'))
print(similar_str(s4[0],s4[1],'lev'))
print(similar_str(s4[0],s4[1],'custom'))

0.25
0.4666666666666666
-----------------------------------------
0.06
0.4666666666666666
-----------------------------------------
0.33
0.099
-----------------------------------------
0.3
0.09
-----------------------------------------
0.79
0.8045
0.74
0.222


In [144]:
def merge(dfa,dfb,pref1='_df1',pref2='_df2',laptoparena_position=None,
          min_score = 0.55, algo = 'lev'):

    df1 = dfa.copy()
    df2 = dfb.copy()

    df1['Index'+pref1]=df1.index
    df2['Index'+pref2]=df2.index

    df1['match_index'+pref2] = np.nan
    df1['match_score'+pref2] = np.nan
    
    choices = []
    for i1,r1 in df1.iterrows():
        score = df2['Series'].apply(similar_str,args=(r1['Series'],algo))
        match_df2 = df2.loc[
            (df2['ram_size'] == r1['ram_size']) &
            (df2['cpu_brand'] == r1['cpu_brand']) &
            (df2['cpu_code'] == r1['cpu_code']) &
            (df2['Brand'] == r1['Brand']) &
            (df2['storage_size'] == r1['storage_size']) &
            (df2['displ_rate']  == r1['displ_rate']) &
            (abs(r1['displ_size']-df2['displ_size'])<0.5) &
            (score > min_score)
        ]
        choices.extend(list(zip(*[[i1]*len(match_df2),match_df2.index,score.loc[match_df2.index]])))

    exclude1, exclude2 = [], []
    for i1,i2,score in sorted(choices,key=lambda x:x[2],reverse=True):
        if i1 not in exclude1 and i2 not in exclude2:
            df1['match_index'+pref2][i1] = i2
            df1['match_score'+pref2][i1] = score
            exclude1.append(i1)
            exclude2.append(i2)
    
    print(f"Matched: {len(df1['match_index'+pref2][np.invert(df1['match_index'+pref2].isnull())])}")

    merged = pd.merge(df1, df2, suffixes=[pref1,pref2], 
                      left_on='match_index'+pref2, right_index = True, 
                      how = laptoparena_position if bool(laptoparena_position) else 'outer'
                      )
    merged.reset_index(inplace=True)
    return merged.reindex(sorted(merged.columns), axis=1)

def join_column_from_merged(df, pref1, pref2,priority='left',keep_cols=[]):
    merged_retained = []
    for c in df.columns:
        # print(c)
        if any(k+'_' in c for k in retained_cols + keep_cols):
            merged_retained.append(c)
    print(merged_retained)
    result = df[merged_retained]

    # to_merge = mapping_cols + other_cols
    to_merge = all_cols
    for c in to_merge:
        result[c] = np.nan
        for i,r in result.iterrows():
            if pd.isnull(df[c+pref1][i]):
                result[c][i] = df[c+pref2][i]
            elif pd.isnull(df[c+pref2][i]):
                result[c][i] = df[c+pref1][i]
            elif priority == 'left':
                result[c][i] = df[c+pref1][i]
            elif priority == 'right':
                result[c][i] = df[c+pref2][i]
    return result

def join_retained_atrr(dataframe, prefs):
    df = dataframe.copy()
    df['Availabe_in'] = len(prefs) - df[['Index'+p for p in prefs]].isnull().sum(axis=1)
    for i,r in df.iterrows():
        r.Rating_count = np.sum(r[['Rating_count'+p for p in prefs]])
        if r.Rating_count ==0:
            r.Rating = 0
        else:
            r.Rating = np.average(r[['Rating'+p for p in prefs]].fillna(0),
                                weights = r[['Rating_count'+p for p in prefs]].fillna(0))
        r.Comment_count = np.sum(r[['Comment_count'+p for p in prefs]])
        r.Price = np.mean(r[['Price'+p for p in prefs]])
        df.loc[i] = r

    for c in retained_cols:
        with pd.option_context('display.max_columns', None):
            print(df[[c]+[c+p for p in prefs]])

    for c in df.columns:
        if any(k+'_' in c for k in retained_cols + ['Index'+p for p in prefs]):
            df = df.drop(columns=c)
    return df

In [146]:
df = merge(cleaned_fpt, cleaned_tgdd,'_fpt','_tgdd',algo='custom')
# df = merge(cleaned_tgdd, cleaned_fpt,'_tgdd','_fpt')
df_2 = join_column_from_merged(df,'_fpt','_tgdd',priority='right',
                               keep_cols=['Index','Series','match_score'])
df_3 = merge(df_2,cleaned_cellphones,'_fpt_tgdd','_cellphones',algo='custom')
df_4 = join_column_from_merged(df_3,'_fpt_tgdd','_cellphones',priority='left',keep_cols=['Index'])
df_4.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57:

Matched: 55
['Comment_count_fpt', 'Comment_count_tgdd', 'Index_fpt', 'Index_tgdd', 'Price_fpt', 'Price_tgdd', 'Rating_count_fpt', 'Rating_count_tgdd', 'Rating_fpt', 'Rating_tgdd', 'Series_fpt', 'Series_tgdd', 'match_score_tgdd']


/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1009: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc[key] = value
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages

Matched: 112
['Comment_count_cellphones', 'Comment_count_fpt', 'Comment_count_fpt_tgdd', 'Comment_count_tgdd', 'Index_cellphones', 'Index_fpt', 'Index_fpt_tgdd', 'Index_tgdd', 'Price_cellphones', 'Price_fpt', 'Price_fpt_tgdd', 'Price_tgdd', 'Rating_cellphones', 'Rating_count_cellphones', 'Rating_count_fpt', 'Rating_count_fpt_tgdd', 'Rating_count_tgdd', 'Rating_fpt', 'Rating_fpt_tgdd', 'Rating_tgdd']


/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1009: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc[key] = value


,Comment_count_cellphones,Comment_count_fpt,Comment_count_fpt_tgdd,Comment_count_tgdd,Index_cellphones,Index_fpt,Index_fpt_tgdd,Index_tgdd,Price_cellphones,Price_fpt,Price_fpt_tgdd,Price_tgdd,Rating_cellphones,Rating_count_cellphones,Rating_count_fpt,Rating_count_fpt_tgdd,Rating_count_tgdd,Rating_fpt,Rating_fpt_tgdd,Rating_tgdd,Comment_count,Price,Rating,Rating_count,Series,Brand,cpu_brand,cpu_code,ram_size,storage_size,displ_size,displ_rate,Battery,GPU,OS,P/N,displ_resolution,hdmi,Release_year,Surface,Weight,backlit_keyboard,lan,length,ram_upgradable,screen_technology,sd_card,storage_extra_slot,storage_type,headphone_jack,thickness,type_c,usb_a,width
0,44.0,598.0,264.0,264.0,https://cellphones.com.vn/laptop-gaming-acer-n...,0.0,0.0,https://www.thegioididong.com/laptop/acer-nitr...,27390000.0,28499000.0,29990000.0,29990000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,264.0,29990000.0,0.0,0.0,nitro 5 gaming an515 57 74nu nh.qd9sv.001,acer,intel,11800h,8.0,512.0,15.6,144.0,57,nvidia geforce rtx3050ti,windows 10,nh.qd9sv.001,1920 x 1080,yes,2021,plastic,2.2,yes,yes,363.4,yes,ips,no,yes,ssd,yes,23.9,yes,yes,255
1,7.0,340.0,53.0,53.0,https://cellphones.com.vn/laptop-msi-modern-14...,1.0,1.0,https://www.thegioididong.com/laptop/msi-gamin...,21990000.0,18799000.0,22990000.0,22990000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,22990000.0,0.0,0.0,gaming modern 14 b11sbu 669vn,msi,intel,1155g7,8.0,512.0,14.0,60.0,39,nvidia geforce mx450,windows 10,669vn,1920 x 1080,yes,2021,metal,1.3,yes,no,319,yes,ips,yes,no,ssd,yes,16.9,yes,yes,219
2,25.0,116.0,730.0,730.0,https://cellphones.com.vn/laptop-dell-inspiron...,2.0,2.0,https://www.thegioididong.com/laptop/dell-insp...,14650000.0,15299000.0,14990000.0,14990000.0,0.0,0.0,0.0,10.0,10.0,0.0,3.3,3.3,730.0,14990000.0,3.3,10.0,inspiron 3501 p90f005n3501c,dell,intel,1115g4,4.0,256.0,15.6,60.0,42,intel uhd graphics,windows 10,p90f005n3501c,1920 x 1080,yes,2020,plastic,1.96,no,yes,363.96,yes,va,yes,no,ssd,yes,19.9,no,yes,249
3,26.0,788.0,788.0,NaN,https://cellphones.com.vn/laptop-dell-vostro-3...,3.0,3.0,NaN,15190000.0,15599000.0,15599000.0,NaN,0.0,0.0,2.0,2.0,NaN,5.0,5.0,NaN,788.0,15599000.0,5.0,2.0,vostro v3500 v5i3001w,dell,intel,1115g4,8.0,256.0,15.6,60.0,45 w,intel uhd graphics,windows 10,v5i3001w,1920 x 1080,yes,2021,nhựa,1.98,yes,yes,368,yes,anti-glare led-backlit,yes,yes,ssd,yes,19.9,yes,yes,249
4,NaN,378.0,378.0,NaN,NaN,4.0,4.0,NaN,NaN,14999000.0,14999000.0,NaN,NaN,NaN,4.0,4.0,NaN,5.0,5.0,NaN,378.0,14999000.0,5.0,4.0,ideapad slim 3 15itl6 82h80040vn,lenovo,intel,1115g4,8.0,512.0,15.6,60.0,--,intel uhd graphics,windows 10,82h80040vn,1920 x 1080,yes,2021,abs plastic,1.65,yes,yes,359,yes,anti-glare led-backlit,yes,yes,ssd,yes,19.9,yes,yes,237.7


In [148]:
df_5 = join_retained_atrr(df_4,('_fpt','_tgdd','_cellphones'))
df_5.head()

     Comment_count  Comment_count_fpt  Comment_count_tgdd  \
0            906.0              598.0               264.0   
1            400.0              340.0                53.0   
2            871.0              116.0               730.0   
3            814.0              788.0                 NaN   
4            378.0              378.0                 NaN   
..             ...                ...                 ...   
571           39.0                NaN                 NaN   
572           21.0                NaN                 NaN   
573           76.0                NaN                 NaN   
574           49.0                NaN                 NaN   
575           29.0                NaN                 NaN   

     Comment_count_cellphones  
0                        44.0  
1                         7.0  
2                        25.0  
3                        26.0  
4                         NaN  
..                        ...  
571                      39.0  
572        

,Index_cellphones,Index_fpt,Index_tgdd,Comment_count,Price,Rating,Series,Brand,cpu_brand,cpu_code,ram_size,storage_size,displ_size,displ_rate,Battery,GPU,OS,P/N,displ_resolution,hdmi,Release_year,Surface,Weight,backlit_keyboard,lan,length,ram_upgradable,screen_technology,sd_card,storage_extra_slot,storage_type,headphone_jack,thickness,type_c,usb_a,width,Availabe_in
0,https://cellphones.com.vn/laptop-gaming-acer-n...,0.0,https://www.thegioididong.com/laptop/acer-nitr...,906.0,2.862633e+07,0.0,nitro 5 gaming an515 57 74nu nh.qd9sv.001,acer,intel,11800h,8.0,512.0,15.6,144.0,57,nvidia geforce rtx3050ti,windows 10,nh.qd9sv.001,1920 x 1080,yes,2021,plastic,2.2,yes,yes,363.4,yes,ips,no,yes,ssd,yes,23.9,yes,yes,255,3
1,https://cellphones.com.vn/laptop-msi-modern-14...,1.0,https://www.thegioididong.com/laptop/msi-gamin...,400.0,2.125967e+07,0.0,gaming modern 14 b11sbu 669vn,msi,intel,1155g7,8.0,512.0,14.0,60.0,39,nvidia geforce mx450,windows 10,669vn,1920 x 1080,yes,2021,metal,1.3,yes,no,319,yes,ips,yes,no,ssd,yes,16.9,yes,yes,219,3
2,https://cellphones.com.vn/laptop-dell-inspiron...,2.0,https://www.thegioididong.com/laptop/dell-insp...,871.0,1.497967e+07,3.3,inspiron 3501 p90f005n3501c,dell,intel,1115g4,4.0,256.0,15.6,60.0,42,intel uhd graphics,windows 10,p90f005n3501c,1920 x 1080,yes,2020,plastic,1.96,no,yes,363.96,yes,va,yes,no,ssd,yes,19.9,no,yes,249,3
3,https://cellphones.com.vn/laptop-dell-vostro-3...,3.0,NaN,814.0,1.539450e+07,5.0,vostro v3500 v5i3001w,dell,intel,1115g4,8.0,256.0,15.6,60.0,45 w,intel uhd graphics,windows 10,v5i3001w,1920 x 1080,yes,2021,nhựa,1.98,yes,yes,368,yes,anti-glare led-backlit,yes,yes,ssd,yes,19.9,yes,yes,249,2
4,NaN,4.0,NaN,378.0,1.499900e+07,5.0,ideapad slim 3 15itl6 82h80040vn,lenovo,intel,1115g4,8.0,512.0,15.6,60.0,--,intel uhd graphics,windows 10,82h80040vn,1920 x 1080,yes,2021,abs plastic,1.65,yes,yes,359,yes,anti-glare led-backlit,yes,yes,ssd,yes,19.9,yes,yes,237.7,1


In [147]:
def print_merged_result(df, col, prefs=('_fpt','_tgdd','_cellphones'),sort=True):
    '''prefs can be any length'''
    print(f"-------Column: {col} ---------------")
    if sort:
        df = df.iloc[df[[col+p for p in prefs]].fillna('').agg(''.join, axis=1).argsort()]
    for i, r in df.iterrows():
        # if abs(r[col+pref1]-r[col+pref2]) >=0.5:
        for p in prefs[1:]:
            print(f"{r['match_score'+p]:4.2f} |",end=" ")
        for p in prefs:
            print(f"{r[col+p]:45} |",end=" ")
        for p in prefs:
            print(f"{r['Index'+p]:80} |",end=" ")
        print("")
# {r[col+pref1]:45} | {r[col+pref2]:40} | {r[col+pref3]:40} | {r['Index'+pref1]:10} | {r['Index'+pref2]:80} | {r['Index'+pref3]:80}")


# print_merged_result(df,col='Series')
print_merged_result(df_3,col='Series',sort=True)


-------Column: Series ---------------
 nan |  nan |                                           nan |                                           nan | 14-cf2033wm 3v7g4ua                           |                                                                              nan |                                                                              nan | https://cellphones.com.vn/laptop-hp-14-cf2033wm-3v7g4ua-pent.html                | 
 nan |  nan |                                           nan |                                           nan | 14-dq2055wm 39k15ua                           |                                                                              nan |                                                                              nan | https://cellphones.com.vn/laptop-hp-14-dq2055wm-39k15ua.html                     | 
 nan |  nan |                                           nan |                                           nan | 14s-dq2544tu 46m22pa                

In [ ]:
s1 = ('yoga slim 7 14itl05 82a300dpvn','yoga 7 14acn6 82n7002lvn')
string_simil

## Join merged result with LaptopArena
Differences when merging with LaptopArena compared to previous part
- Condition of `Series` will be different
- df1 and df2 has different set of columns
- Mapping column may contains `NaN`


In [149]:
df_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 37 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Index_cellphones    416 non-null    object 
 1   Index_fpt           146 non-null    float64
 2   Index_tgdd          181 non-null    object 
 3   Comment_count       576 non-null    float64
 4   Price               520 non-null    float64
 5   Rating              576 non-null    float64
 6   Series              576 non-null    object 
 7   Brand               576 non-null    object 
 8   cpu_brand           576 non-null    object 
 9   cpu_code            576 non-null    object 
 10  ram_size            576 non-null    float64
 11  storage_size        576 non-null    float64
 12  displ_size          576 non-null    float64
 13  displ_rate          576 non-null    float64
 14  Battery             522 non-null    object 
 15  GPU                 573 non-null    object 
 16  OS      

In [80]:
text_facet(df_5,'Series')

---------- Series ------------
imac 24 2021 m1 i                                        5
macbook pro m1 2020                                      4
imac 27 2020                                             3
macbook air m1 2020                                      3
surface pro 7 2019                                       2
gaming g5 5510                                           2
macbook pro 13 touch bar 2020 i                          2
macbook pro 13 touch bar m1 2020 i                       2
surface laptop 3                                         2
macbook pro 2021 m1 pro                                  2
ideapad 3                                                2
vivobook flip tp412fa                                    2
modern 14 b11mo                                          2
ideapad 3 14alc6                                         2
vivobook s433ea                                          2
macbook pro 2021                                         2
modern 14 b11sb          

In [ ]:
print(len(df_5))
df_5.to_csv('merged_data.csv')

In [ ]:
df_a = merge(df_5,cleaned_laptoparena,pref1='_vnstore',pref2='_laptoparena',
             laptoparena_position='left',min_score=0.55,algo='custom')

In [27]:
print(len(df_a))
print_merged_result(df_a,col='Series',prefs=('_vnstore','_laptoparena'))

552
-------Column: Series ---------------
 nan | 14-cf2033wm 3v7g4ua                           |                                           nan |                                                                              428 |                                                                              nan | 
 nan | 14-dq2055wm 39k15ua                           |                                           nan |                                                                              429 |                                                                              nan | 
 nan | 14s-dq2544tu 46m22pa                          |                                           nan |                                                                              430 |                                                                              nan | 
 nan | 14s-fq1066au 4k0z6pa                          |                                           nan |                                              

In [29]:
text_facet(df_a,'Release_date')

---------- Release_date ------------
NaT           329
2021-01-13     10
2020-10-06      7
2020-11-22      6
2020-12-25      5
2020-12-19      5
2020-11-19      4
2020-10-22      4
2021-01-25      4
2020-11-18      3
2021-04-16      3
2020-10-26      3
2021-01-16      3
2021-03-22      3
2021-01-19      3
2021-03-10      3
2020-12-04      3
2020-12-28      3
2020-11-06      3
2020-12-13      2
2021-03-04      2
2021-04-29      2
2021-03-13      2
2020-07-25      2
2020-11-26      2
2021-03-26      2
2020-12-22      2
2021-05-04      2
2021-03-19      2
2021-02-10      2
2020-11-04      2
2021-02-04      2
2021-01-22      2
2019-09-19      2
2021-06-09      2
2021-07-31      2
2020-06-25      2
2020-05-12      2
2021-05-13      2
2021-01-04      2
2020-07-19      2
2020-05-22      2
2020-09-19      2
2020-11-13      2
2020-12-07      2
2020-06-04      2
2021-01-31      2
2021-02-11      2
2020-07-05      1
2020-10-10      1
2021-01-07      1
2021-03-28      1
2020-06-01      1
2020-04-1